<a href="https://colab.research.google.com/github/aneetalr/BirdSoundClassification/blob/main/audio_preprocess_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import matplotlib
matplotlib.interactive(False)
matplotlib.use('Agg')
import os
import gc

In [ ]:
basePath="/content/drive/MyDrive/DatasetBird"
melsPath= "/content/drive/MyDrive/melspec-final/"

In [ ]:
birds=[] # list of all birds
for root, dirs, files in os.walk(basePath):
    if root == basePath:
        birds=dirs
print(birds,end="\n\n")

birds50=[]                
flist=[] # list of all files
blist=[] # list of files for one bird 
i50=0;
for i, bird in enumerate(birds):
    print(i,bird)
    for root, dirs, files in os.walk(basePath+"/"+bird):
        print(files)
        for file in files:
            if file.endswith(".mp3"):
                blist.append(os.path.join(root, file))
    i50 = i50+1;
    print(i50, ". Found ", len(blist), ' files for ', bird ,'(',i+1,')')
    birds50.append(bird)
    flist.append(blist)
    blist=[]
    print()
print(birds50) 
print(root)

['comter', 'amered', 'mallar3', 'blujay', 'bkcchi']

0 comter
['XC120379.mp3', 'XC134798.mp3', 'XC133649.mp3', 'XC134786.mp3', 'XC138784.mp3', 'XC134724.mp3', 'XC129657.mp3', 'XC138762.mp3', 'XC120430.mp3', 'XC145339.mp3', 'XC145754.mp3', 'XC161067.mp3', 'XC141283.mp3', 'XC141280.mp3', 'XC161072.mp3', 'XC161069.mp3', 'XC182743.mp3', 'XC182899.mp3', 'XC138866.mp3', 'XC141284.mp3', 'XC161071.mp3', 'XC161068.mp3', 'XC161070.mp3', 'XC182782.mp3', 'XC141279.mp3', 'XC180814.mp3', 'XC182897.mp3', 'XC183234.mp3', 'XC145755.mp3', 'XC182898.mp3', 'XC184889.mp3', 'XC191411.mp3', 'XC190510.mp3', 'XC247657.mp3', 'XC244403.mp3', 'XC190182.mp3', 'XC189342.mp3', 'XC191409.mp3', 'XC25566.mp3', 'XC247952.mp3', 'XC189344.mp3', 'XC187357.mp3', 'XC191410.mp3', 'XC184892.mp3', 'XC200363.mp3', 'XC189010.mp3', 'XC190511.mp3', 'XC189345.mp3', 'XC184446.mp3', 'XC184447.mp3', 'XC25691.mp3', 'XC263276.mp3', 'XC263158.mp3', 'XC266251.mp3', 'XC266250.mp3', 'XC272306.mp3', 'XC266248.mp3', 'XC272309.mp3', 'XC272308.m

In [ ]:
def saveMel(signal, directory):
    gc.enable()
    # MK_spectrogram modified
    N_FFT = 1024         # 
    HOP_SIZE = 1024      #  
    N_MELS = 128          # Higher   
    WIN_SIZE = 1024      # 
    WINDOW_TYPE = 'hann' # 
    FEATURE = 'mel'      # 
    FMIN = 1400

    fig = plt.figure(1,frameon=False)
    fig.set_size_inches(6,6)

    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    
    S = librosa.feature.melspectrogram(y=signal, sr=sr,
                                        n_fft=N_FFT,
                                        hop_length=HOP_SIZE, 
                                        n_mels=N_MELS, 
                                        htk=True, 
                                        fmin=FMIN, # higher limit ##high-pass filter freq.
                                        fmax=sr/2) # AMPLITUDE
    librosa.display.specshow(librosa.power_to_db(S**2,ref=np.max), fmin=FMIN) #power = S**2
    
    fig.savefig(directory)
    plt.ioff()
    #plt.show(block=False)
    fig.clf()
    ax.cla()
    plt.clf()
    plt.close('all')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import sys
from tqdm import tqdm_notebook as tqdm
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

size = {'desired': 10, # [seconds]
        'minimum':  5, # [seconds]
        'stride' : 1, # [seconds]
        'name': 5 # [number of letters]
       } # stride should not be bigger than desired length

print('Number of directories to check and cut: ', len(flist))

#step = (size['desired']-size['stride'])*sr # length of step between two cuts in seconds
step=1
if step>0:
    for bird, birdList in enumerate(flist):
        print("Processing ",bird,'. ', birds50[bird], "...")
        for birdnr, path in tqdm(enumerate(birdList)):
            # load the mp3 file
            directory=melsPath+str(bird)+birds50[bird][:size['name']]+"/"

            if not os.path.exists(directory):
                os.makedirs(directory)
                
            if not os.path.exists(directory+path.rsplit('/',1)[1].replace(' ', '')[:-4]+"1_1.png"):
                  
                signal, sr = librosa.load(path) # sr = sampling rate
                step = (size['desired']-size['stride'])*sr # length of step between two cuts in seconds
                
                nr=0;
                for start, end in zip(range(0,len(signal),step),range(size['desired']*sr,len(signal),step)):
                    # cut file and save each piece
                    nr=nr+1
                    # save the file if its length is higher than minimum
                    if end-start > size['minimum']*sr:
                        melpath=path.rsplit('/',1)[1]
                        melpath=directory+melpath.replace(' ', '')[:-4]+str(nr)+"_"+str(nr)+".png"
                        saveMel(signal[start:end],melpath)
                    #print('New file...',start/sr,' - ',end/sr)
                    #print('Start: ',start,'end: ', end, 'length: ', end-start)
                
            pass
else:    
    print("Error: Stride should be lower than desired length.")
    
print('Number of files after cutting: ')

Number of directories to check and cut:  5
Processing  0 .  comter ...


0it [00:00, ?it/s]

Processing  1 .  amered ...


0it [00:00, ?it/s]

Processing  2 .  mallar3 ...


...

Processing  3 .  blujay ...


0it [00:00, ?it/s]

Processing  4 .  bkcchi ...


0it [00:00, ?it/s]

Number of files after cutting: 


In [ ]:
for sub in os.listdir(melsPath):
  print(sub,' - ',len(os.listdir(melsPath+'/'+sub)))

0comte  -  787
1amere  -  988
2malla  -  956
3bluja  -  908
4bkcch  -  948
